In [1]:
# SIPP Portfolio Performance.
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Parameters.
portfolio_name = 'SIPP-G'

In [3]:
# Import portfolio equity holdings.
dh = pd.read_csv(
    f'{portfolio_name}_holdings.csv',
    header=0,
    names=[
        'date',
        'name',
        'tidm',
        'shares',
        'price',
        'cost',
        'charges',
        'sduty',
        'value',
        'unrealised',
        'profit',
        'pct_return',
        'pct_annual',
    ],
    index_col=0,
    usecols=['date', 'name', 'tidm', 'profit'],
    engine='python',
    skipfooter=1,
    parse_dates=True,
    dayfirst=True,
)

# Filter portfolio equity holdings for closed positions.
dh = dh[dh.name == 'Position Closed']
dh = dh.sort_index()
dh

,name,tidm,profit
date,,,
2018-08-14,Position Closed,NXR,221.25
2018-12-11,Position Closed,GFRD,-406.77
2018-12-20,Position Closed,CMS#1,510.83
2019-04-09,Position Closed,PFC,0.13
2019-07-09,Position Closed,IAG,-293.78
...,...,...,...
2022-04-01,Position Closed,RNWH,9.87
2022-04-19,Position Closed,BRCK,243.59
2022-04-19,Position Closed,WOSG,2356.99


In [4]:
# Import portfolio cash transactions.
dt = pd.read_csv(
    f'{portfolio_name}_transactions.csv',
    header=0,
    names=[
        'date',
        'time',
        'ionic_id',
        'ticker',
        'type',
        'shares',
        'price',
        'broker',
        'stamp',
        'cash',
        'non_cash',
        'note'
    ], 
    index_col=0,
    usecols=['date', 'type', 'cash'],
    parse_dates=True,
    dayfirst=True,
)

# Filter portfolio cash transactions for credits, interest, & charges.
dt = dt[(dt.type == 'Credit') | (dt.type == 'Interest') | (dt.type == 'Charges')]
dt = dt.sort_index()
dt

,type,cash
date,,
2017-05-24,Credit,1079.16
2017-06-09,Interest,0.02
2017-07-04,Charges,0.38
2017-07-05,Credit,550.00
2017-07-09,Interest,0.01
...,...,...
2022-05-09,Credit,2.91
2022-05-10,Credit,892.32
2022-06-07,Charges,11.13


In [5]:
# Total portfolio value (closed equity holdings + cash transactions)
start_date = min(dh.index[0], dt.index[0])
end_date = max(dh.index[-1], dt.index[-1])
date_range = pd.date_range(start=start_date, end=end_date)
date_range

DatetimeIndex(['2017-05-24', '2017-05-25', '2017-05-26', '2017-05-27',
               '2017-05-28', '2017-05-29', '2017-05-30', '2017-05-31',
               '2017-06-01', '2017-06-02',
               ...
               '2022-06-11', '2022-06-12', '2022-06-13', '2022-06-14',
               '2022-06-15', '2022-06-16', '2022-06-17', '2022-06-18',
               '2022-06-19', '2022-06-20'],
              dtype='datetime64[ns]', length=1854, freq='D')

In [6]:
portfolio = pd.concat([dh.profit, dt.cash])
portfolio = portfolio.sort_index()
portfolio

date
2017-05-24    1079.16
2017-06-09       0.02
2017-07-04       0.38
2017-07-05     550.00
2017-07-09       0.01
               ...   
2022-05-23    -229.20
2022-06-07      11.13
2022-06-09       1.80
2022-06-15     892.32
2022-06-20   -1255.85
Length: 244, dtype: float64

In [7]:
portfolio = portfolio.groupby(level=0).sum()
portfolio

date
2017-05-24    1079.16
2017-06-09       0.02
2017-07-04       0.38
2017-07-05     550.00
2017-07-09       0.01
               ...   
2022-05-23    -229.20
2022-06-07      11.13
2022-06-09       1.80
2022-06-15     892.32
2022-06-20   -1255.85
Length: 191, dtype: float64

In [8]:
portfolio = portfolio.reindex(date_range, method='ffill') # Do cumsum before ffill 
portfolio

2017-05-24    1079.16
2017-05-25    1079.16
2017-05-26    1079.16
2017-05-27    1079.16
2017-05-28    1079.16
               ...   
2022-06-16     892.32
2022-06-17     892.32
2022-06-18     892.32
2022-06-19     892.32
2022-06-20   -1255.85
Freq: D, Length: 1854, dtype: float64

In [9]:
portfolio.loc['2020-11']

2020-11-01   -1116.20
2020-11-02   -1116.20
2020-11-03     903.58
2020-11-04     903.58
2020-11-05     903.58
2020-11-06     903.58
2020-11-07     903.58
2020-11-08     903.58
2020-11-09     903.58
2020-11-10    -185.40
2020-11-11    -185.40
2020-11-12    -185.40
2020-11-13    -185.40
2020-11-14    -185.40
2020-11-15    -185.40
2020-11-16    -185.40
2020-11-17    -185.40
2020-11-18    -185.40
2020-11-19    -185.40
2020-11-20    -185.40
2020-11-21    -185.40
2020-11-22    -185.40
2020-11-23    -185.40
2020-11-24    -185.40
2020-11-25    -185.40
2020-11-26    -185.40
2020-11-27    -185.40
2020-11-28    -185.40
2020-11-29    -185.40
2020-11-30    -185.40
Freq: D, dtype: float64